In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import EllipticEnvelope
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import albumentations as A
from albumentations.pytorch import ToTensorV2
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# device agnostic code
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# hyperparameters and file path configuration
CONFIG = {
    'img_size': 224,
    'batch_size': 32,
    'epochs': 15,
    'learning_rate': 2e-4,
    'weight_decay': 1e-4,
    'n_folds': 3,
    'seed': 42,
    'model_name': 'efficientnet_b0',
    'early_stopping_patience': 5,
    'contamination': 0.15,
    'reconstruction_weight': 1.0,
    'feature_weight': 0.5,
    'train_images_dir' : "/kaggle/input/soil-classification-part-2/soil_competition-2025/train",
    'test_images_dir' : "/kaggle/input/soil-classification-part-2/soil_competition-2025/test",
    'train_csv_path' : "/kaggle/input/soil-classification-part-2/soil_competition-2025/train_labels.csv",
    'test_csv_path' : "/kaggle/input/soil-classification-part-2/soil_competition-2025/test_ids.csv",
}

In [ ]:
# seed for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(CONFIG['seed'])

In [ ]:
# input data structure
class SoilDataset(Dataset):
    def __init__(self, image_paths, transforms=None, is_test=False):
        self.image_paths = image_paths
        self.transforms = transforms
        self.is_test = is_test

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]

        # Load image
        image = cv2.imread(img_path)
        if image is None:
            image = np.zeros((224, 224, 3), dtype=np.uint8)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Keep original for reconstruction loss
        original_image = image.copy()

        # Apply transforms
        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']
        else:
            # Ensure consistent tensor creation
            image = image.astype(np.float32) / 255.0
            image = torch.from_numpy(image.transpose(2, 0, 1)).contiguous()

        if self.is_test:
            return image
        else:
            # Apply same base transforms to original for consistent dimensions
            if self.transforms:
                # Create a simple transform for original
                original_transform = A.Compose([
                    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
                    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ToTensorV2()
                ])
                original_transformed = original_transform(image=original_image)
                original_tensor = original_transformed['image']
            else:
                original_image = original_image.astype(np.float32) / 255.0
                original_tensor = torch.from_numpy(original_image.transpose(2, 0, 1)).contiguous()
            
            return image, original_tensor

In [ ]:
# data augmentation
def get_transforms(phase):
    if phase == 'train':
        return A.Compose([
            A.Resize(CONFIG['img_size'], CONFIG['img_size']),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.3),
            A.Rotate(limit=30, p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.5),
            A.HueSaturationValue(hue_shift_limit=8, sat_shift_limit=15, val_shift_limit=8, p=0.4),
            A.GaussNoise(var_limit=(5.0, 25.0), p=0.3),
            A.GaussianBlur(blur_limit=3, p=0.2),
            A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.3),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, p=0.4),
            A.CoarseDropout(max_holes=4, max_height=16, max_width=16, p=0.2),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Resize(CONFIG['img_size'], CONFIG['img_size']),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

In [ ]:
# Model initialization
class SoilAutoencoder(nn.Module):
    def __init__(self, model_name='efficientnet_b0', pretrained=True):
        super(SoilAutoencoder, self).__init__()
        
        # Encoder
        if model_name == 'efficientnet_b0':
            self.encoder_backbone = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1 if pretrained else None)
            in_features = self.encoder_backbone.classifier[1].in_features
            self.encoder_backbone.classifier = nn.Identity()
        
        # Feature compression layers
        self.encoder = nn.Sequential(
            nn.Linear(in_features, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, 256),  # Bottleneck layer
        )
        
        # Decoder layers
        self.decoder = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, 3 * CONFIG['img_size'] * CONFIG['img_size']),
            nn.Sigmoid()
        )
        
        #  layers for feature-based anomaly detection
        self.feature_classifier = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),  # Anomaly score
            nn.Sigmoid()
        )
        
    def forward(self, x):
        batch_size = x.size(0)
        
        # Encode
        backbone_features = self.encoder_backbone(x)
        encoded = self.encoder(backbone_features)
        
        # Decode
        decoded = self.decoder(encoded)
        decoded = decoded.view(batch_size, 3, CONFIG['img_size'], CONFIG['img_size'])
        
        # Anomaly score
        anomaly_score = self.feature_classifier(encoded)
        
        return decoded, encoded, anomaly_score

In [ ]:
class OneClassLoss(nn.Module):
    def __init__(self, reconstruction_weight=1.0, feature_weight=0.5, center_weight=0.3):
        super(OneClassLoss, self).__init__()
        self.reconstruction_weight = reconstruction_weight
        self.feature_weight = feature_weight
        self.center_weight = center_weight
        self.mse_loss = nn.MSELoss()
        self.center = None
        
    def update_center(self, features):
        """Update the center of normal features"""
        if self.center is None:
            self.center = torch.mean(features, dim=0)
        else:
            # Moving average
            self.center = 0.9 * self.center + 0.1 * torch.mean(features, dim=0)
    
    def forward(self, reconstructed, original, features, anomaly_scores):
        # Reconstruction loss
        recon_loss = self.mse_loss(reconstructed, original)
        
        # Feature-based loss, encourage normal samples to have low anomaly scores
        feature_loss = torch.mean(anomaly_scores)
        
        # Center loss, encourage features to be close to center
        if self.center is not None:
            center_loss = torch.mean(torch.sum((features - self.center.detach()) ** 2, dim=1))
        else:
            center_loss = torch.tensor(0.0, device=features.device)
        
        total_loss = (self.reconstruction_weight * recon_loss + 
                     self.feature_weight * feature_loss +
                     self.center_weight * center_loss)
        
        return total_loss, recon_loss, feature_loss, center_loss

In [ ]:
# training loop
def train_epoch(model, dataloader, criterion, optimizer, device, epoch):
    model.train()
    running_loss = 0.0
    running_recon_loss = 0.0
    running_feature_loss = 0.0
    running_center_loss = 0.0

    for batch_idx, (images, originals) in enumerate(dataloader):
        images, originals = images.to(device, non_blocking=True), originals.to(device, non_blocking=True)

        optimizer.zero_grad()

        # Forward pass
        reconstructed, features, anomaly_scores = model(images)

        # Update center
        criterion.update_center(features)

        # Calculate loss
        total_loss, recon_loss, feature_loss, center_loss = criterion(
            reconstructed, originals, features, anomaly_scores)

        # Backward pass
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        # Statistics
        running_loss += total_loss.item()
        running_recon_loss += recon_loss.item()
        running_feature_loss += feature_loss.item()
        running_center_loss += center_loss.item()

        # Clear cache periodically
        if batch_idx % 50 == 0:
            torch.cuda.empty_cache()

    epoch_loss = running_loss / len(dataloader)
    epoch_recon_loss = running_recon_loss / len(dataloader)
    epoch_feature_loss = running_feature_loss / len(dataloader)
    epoch_center_loss = running_center_loss / len(dataloader)

    return epoch_loss, epoch_recon_loss, epoch_feature_loss, epoch_center_loss

In [ ]:
# validation loop
def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_recon_loss = 0.0
    running_feature_loss = 0.0
    running_center_loss = 0.0

    with torch.no_grad():
        for images, originals in dataloader:
            images, originals = images.to(device), originals.to(device)

            reconstructed, features, anomaly_scores = model(images)
            total_loss, recon_loss, feature_loss, center_loss = criterion(
                reconstructed, originals, features, anomaly_scores)

            running_loss += total_loss.item()
            running_recon_loss += recon_loss.item()
            running_feature_loss += feature_loss.item()
            running_center_loss += center_loss.item()

    epoch_loss = running_loss / len(dataloader)
    epoch_recon_loss = running_recon_loss / len(dataloader)
    epoch_feature_loss = running_feature_loss / len(dataloader)
    epoch_center_loss = running_center_loss / len(dataloader)

    return epoch_loss, epoch_recon_loss, epoch_feature_loss, epoch_center_loss

In [ ]:
def prepare_data():
    # Load training data
    train_df = pd.read_csv(CONFIG['train_csv_path'])
    print(f"Loaded train.csv with {len(train_df)} rows")

    train_images = []

    image_col = 'image_id'
    if image_col not in train_df.columns:
        image_col = train_df.columns[0]

    for idx, row in train_df.iterrows():
        image_name = str(row[image_col])

        for ext in ['', '.jpg', '.jpeg', '.png', '.bmp']:
            img_path = os.path.join(CONFIG['train_images_dir'], image_name + ext)
            if os.path.exists(img_path):
                train_images.append(img_path)
                break
        else:
            img_path = os.path.join(CONFIG['train_images_dir'], image_name)
            if os.path.exists(img_path):
                train_images.append(img_path)

    print(f"Found {len(train_images)} valid training images")
    return train_images

In [ ]:
def extract_features(model, dataloader, device):
    model.eval()
    features_list = []
    anomaly_scores_list = []
    recon_errors_list = []
    
    with torch.no_grad():
        for batch in dataloader:
            if isinstance(batch, tuple):
                images, originals = batch
                originals = originals.to(device)
            else:
                images = batch
                originals = None
            
            images = images.to(device)
            reconstructed, features, anomaly_scores = model(images)
            
            features_list.append(features.cpu().numpy())
            anomaly_scores_list.append(anomaly_scores.cpu().numpy())
            
            if originals is not None:
                recon_error = torch.mean((reconstructed - originals) ** 2, dim=[1, 2, 3])
                recon_errors_list.append(recon_error.cpu().numpy())
    
    features = np.vstack(features_list)
    anomaly_scores = np.vstack(anomaly_scores_list).flatten()
    recon_errors = np.concatenate(recon_errors_list) if recon_errors_list else None
    
    return features, anomaly_scores, recon_errors

In [ ]:
def train_one_class_model():
    # Prepare data
    train_images = prepare_data()

    if len(train_images) == 0:
        print("No training images found!")
        return None, []

    # K-Fold Cross Validation
    kf = KFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=CONFIG['seed'])
    fold_models = []
    fold_losses = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(train_images)):
        print(f"\n{'='*50}")
        print(f"FOLD {fold + 1}/{CONFIG['n_folds']}")
        print(f"{'='*50}")

        # Split data
        train_imgs = [train_images[i] for i in train_idx]
        val_imgs = [train_images[i] for i in val_idx]

        # Create datasets
        train_dataset = SoilDataset(train_imgs, get_transforms('train'))
        val_dataset = SoilDataset(val_imgs, get_transforms('val'))

        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'],
                                shuffle=True, num_workers=2, pin_memory=True, 
                                persistent_workers=True, drop_last=True)
        val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'],
                              shuffle=False, num_workers=2, pin_memory=True,
                              persistent_workers=True)

        # Initialize model
        model = SoilAutoencoder(CONFIG['model_name'], pretrained=True)
        model.to(device)

        # Initialize optimizer
        optimizer = optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'],
                               weight_decay=CONFIG['weight_decay'])
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['epochs'])

        # Initialize loss function
        criterion = OneClassLoss(
            reconstruction_weight=CONFIG['reconstruction_weight'],
            feature_weight=CONFIG['feature_weight']
        )

        # Training loop
        best_loss = float('inf')
        patience_counter = 0

        train_losses = []
        val_losses = []

        for epoch in range(CONFIG['epochs']):
            # Train
            train_loss, train_recon, train_feature, train_center = train_epoch(
                model, train_loader, criterion, optimizer, device, epoch)

            # Validate
            val_loss, val_recon, val_feature, val_center = validate_epoch(
                model, val_loader, criterion, device)

            scheduler.step()

            train_losses.append(train_loss)
            val_losses.append(val_loss)

            print(f'Epoch {epoch+1:02d}/{CONFIG["epochs"]:02d} | '
                  f'Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | '
                  f'Recon: {val_recon:.4f} | Feature: {val_feature:.4f}')

            # Save best model
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), f'best_autoencoder_fold_{fold+1}.pth')
                patience_counter = 0
            else:
                patience_counter += 1

            # Early stopping
            if patience_counter >= CONFIG['early_stopping_patience']:
                print(f'Early stopping at epoch {epoch+1}')
                break

        # Load best model
        model.load_state_dict(torch.load(f'best_autoencoder_fold_{fold+1}.pth'))
        fold_models.append(model)
        fold_losses.append(best_loss)

        print(f'Fold {fold+1} Best Loss: {best_loss:.4f}')

        # Plot training curves
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.plot(train_losses, label='Train Loss')
        plt.plot(val_losses, label='Val Loss')
        plt.title(f'Fold {fold+1} - Training Curves')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.show()

        # Clear memory
        del train_loader, val_loader
        torch.cuda.empty_cache()

    print(f"\nCross-validation completed!")
    print(f"Fold losses: {fold_losses}")
    print(f"Mean loss: {np.mean(fold_losses):.4f} ± {np.std(fold_losses):.4f}")

    return fold_models, fold_losses

In [ ]:
# Predict anomalies using ensemble of methods
def predict_anomalies(models, test_loader, train_loader, device):
    # 1. Extract features from all models
    all_test_features = []
    all_test_anomaly_scores = []
    all_test_recon_errors = []

    for model in models:
        test_features, test_anomaly_scores, test_recon_errors = extract_features(
            model, test_loader, device)
        all_test_features.append(test_features)
        all_test_anomaly_scores.append(test_anomaly_scores)
        if test_recon_errors is not None:
            all_test_recon_errors.append(test_recon_errors)

    # Average across models
    avg_test_features = np.mean(all_test_features, axis=0)
    avg_test_anomaly_scores = np.mean(all_test_anomaly_scores, axis=0)
    avg_test_recon_errors = np.mean(all_test_recon_errors, axis=0) if all_test_recon_errors else None

    # 2. Get training features for comparison
    train_features_list = []
    for model in models:
        train_features, _, _ = extract_features(model, train_loader, device)
        train_features_list.append(train_features)

    avg_train_features = np.mean(train_features_list, axis=0)

    # 3. Classical anomaly detection methods
    predictions = {}

    # One-Class SVM
    scaler = StandardScaler()
    train_features_scaled = scaler.fit_transform(avg_train_features)
    test_features_scaled = scaler.transform(avg_test_features)

    oc_svm = OneClassSVM(kernel='rbf', gamma='scale', nu=CONFIG['contamination'])
    oc_svm.fit(train_features_scaled)
    svm_pred = oc_svm.predict(test_features_scaled)
    predictions['svm'] = np.where(svm_pred == 1, 1, 0)

    # Isolation Forest
    iso_forest = IsolationForest(contamination=CONFIG['contamination'], random_state=CONFIG['seed'])
    iso_forest.fit(avg_train_features)
    iso_pred = iso_forest.predict(avg_test_features)
    predictions['isolation'] = np.where(iso_pred == 1, 1, 0)

    # Elliptic Envelope
    elliptic = EllipticEnvelope(contamination=CONFIG['contamination'], random_state=CONFIG['seed'])
    elliptic.fit(train_features_scaled)
    elliptic_pred = elliptic.predict(test_features_scaled)
    predictions['elliptic'] = np.where(elliptic_pred == 1, 1, 0)

    # Anomaly score threshold
    anomaly_threshold = np.percentile(avg_test_anomaly_scores, (1 - CONFIG['contamination']) * 100)
    predictions['anomaly_score'] = np.where(avg_test_anomaly_scores <= anomaly_threshold, 1, 0)

    # Reconstruction error threshold
    if avg_test_recon_errors is not None:
        recon_threshold = np.percentile(avg_test_recon_errors, (1 - CONFIG['contamination']) * 100)
        predictions['reconstruction'] = np.where(avg_test_recon_errors <= recon_threshold, 1, 0)

    # Ensemble prediction (majority voting)
    all_preds = np.array(list(predictions.values()))
    ensemble_pred = np.where(np.mean(all_preds, axis=0) >= 0.5, 1, 0)

    return ensemble_pred, predictions

In [ ]:
def predict_test_set(models):
    # Load test data
    test_df = pd.read_csv(CONFIG['test_csv_path'])
    print(f"Loaded test.csv with {len(test_df)} rows")

    test_images = []
    image_col = 'image_id'
    if image_col not in test_df.columns:
        image_col = test_df.columns[0]

    for idx, row in test_df.iterrows():
        image_name = str(row[image_col])
        for ext in ['', '.jpg', '.jpeg', '.png', '.bmp']:
            img_path = os.path.join(CONFIG['test_images_dir'], image_name + ext)
            if os.path.exists(img_path):
                test_images.append(img_path)
                break
        else:
            img_path = os.path.join(CONFIG['test_images_dir'], image_name)
            if os.path.exists(img_path):
                test_images.append(img_path)

    print(f"Found {len(test_images)} test images")
    train_images = prepare_data()
    train_dataset = SoilDataset(train_images, get_transforms('val'))
    test_dataset = SoilDataset(test_images, get_transforms('val'), is_test=True)

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'],
                            shuffle=False, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'],
                           shuffle=False, num_workers=2, pin_memory=True)

    # Make predictions
    ensemble_pred, individual_preds = predict_anomalies(models, test_loader, train_loader, device)

    # Show prediction breakdown
    print("\nPrediction Results:")
    for method, preds in individual_preds.items():
        soil_count = np.sum(preds)
        print(f"{method:15}: Soil={soil_count:4d} ({soil_count/len(preds)*100:.1f}%), "
              f"Non-soil={len(preds)-soil_count:4d} ({(len(preds)-soil_count)/len(preds)*100:.1f}%)")

    soil_count = np.sum(ensemble_pred)
    print(f"{'Ensemble':15}: Soil={soil_count:4d} ({soil_count/len(ensemble_pred)*100:.1f}%), "
          f"Non-soil={len(ensemble_pred)-soil_count:4d} ({(len(ensemble_pred)-soil_count)/len(ensemble_pred)*100:.1f}%)")

    # Create submission
    submission_df = pd.DataFrame({
        'image_id': test_df[image_col],
        'label': ensemble_pred
    })

    submission_df.to_csv('submission.csv', index=False)
    print("\nSubmission file created: submission.csv")

    return submission_df

In [ ]:
if __name__ == "__main__":
    models, fold_losses = train_one_class_model()

    if models:
        # Make predictions
        submission_df = predict_test_set(models)

        # Save best model
        best_model_idx = np.argmin(fold_losses)
        best_model = models[best_model_idx]

        torch.save({
            'model_state_dict': best_model.state_dict(),
            'config': CONFIG,
            'fold_losses': fold_losses,
            'best_fold': best_model_idx + 1,
        }, 'best_one_class_model.pth')

        # Print completion statements
        print(f"\nTraining completed!")
        print(f"Best model from fold {best_model_idx + 1} saved as 'best_one_class_model.pth'")
        print(f"Fold losses: {fold_losses}")
        print(f"Mean loss: {np.mean(fold_losses):.4f} ± {np.std(fold_losses):.4f}")
    else:
        print("Training failed - no models created")